In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from gensim.models import word2vec

from sklearn.manifold import TSNE
import time
from tqdm import tqdm

import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

import nltk
from nltk.corpus import stopwords
import string

from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
print(os.listdir("../input/embeddings"))

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [ ]:
#Size of datasets
print("Size of train data  (Rows, Columns): ",train_df.shape)
print("Size of test data  (Rows, Columns): ",test_df.shape)

In [ ]:
#Snapshot of train data
train_df.head()

In [ ]:
#Sanapshot of test data
test_df.head()

Target Variable Distribution

In [ ]:
np.unique(train_df['target'].values)

In [ ]:
np.mean(train_df['target'].values)

In [ ]:
print('---------------------------------------------------------------------------------')
print(train_df['target'].value_counts())
print('---------------------------------------------------------------------------------')
print(train_df['target'].value_counts()/train_df['target'].shape[0])
print('---------------------------------------------------------------------------------')
#sns.set(style="darkgrid")
ax = sns.countplot(x=train_df['target'], data=train_df)

__Insight:__ 6.1% of the training data are insincere questions and rest of them are sincere.

In [ ]:
eng_stopwords = set(stopwords.words("english"))

In [ ]:
## Number of words in the text ##
train_df["num_words"] = train_df["question_text"].apply(lambda x: len(str(x).split()))
test_df["num_words"] = test_df["question_text"].apply(lambda x: len(str(x).split()))

In [ ]:
cnt_srs = train_df["num_words"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of words in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()




In [ ]:
## Number of unique words in the text ##
train_df["num_unique_words"] = train_df["question_text"].apply(lambda x: len(set(str(x).split())))
test_df["num_unique_words"] = test_df["question_text"].apply(lambda x: len(set(str(x).split())))

In [ ]:

cnt_srs = train_df["num_unique_words"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of num_unique_words in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Number of characters in the text ##
train_df["num_chars"] = train_df["question_text"].apply(lambda x: len(str(x)))
test_df["num_chars"] = test_df["question_text"].apply(lambda x: len(str(x)))

In [ ]:

cnt_srs = train_df["num_chars"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of char in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Number of stopwords in the text ##
train_df["num_stopwords"] = train_df["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df["num_stopwords"] = test_df["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

In [ ]:

cnt_srs = train_df["num_stopwords"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of stopwords in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Number of punctuations in the text ##
train_df["num_punctuations"] =train_df['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test_df["num_punctuations"] =test_df['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

In [ ]:

cnt_srs = train_df["num_punctuations"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of punctuations in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Number of title case words in the text ##
train_df["num_words_upper"] = train_df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df["num_words_upper"] = test_df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [ ]:

cnt_srs = train_df["num_words_upper"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of words_upper in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Number of title case words in the text ##
train_df["num_words_title"] = train_df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_df["num_words_title"] = test_df["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))


In [ ]:

cnt_srs = train_df["num_words_title"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of words_title in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
## Average length of the words in the text ##
train_df["mean_word_len"] = train_df["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df["mean_word_len"] = test_df["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:

cnt_srs = train_df["mean_word_len"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[0])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of mean_word_len in the question', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
features = ['num_words', 'num_unique_words', 'num_chars', 
                'num_stopwords', 'num_punctuations', 'num_words_upper', 
                'num_words_title', 'mean_word_len']

In [ ]:
for i in features:
    plt.figure(figsize=(8,4))
    sns.set(style="whitegrid")
    sns.violinplot(data=train_df[i])
    plt.show()

In [ ]:
def missing_check(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
missing_check(train_df)

In [ ]:
missing_check(test_df)

In [ ]:
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


In [ ]:
train_df['question_text'] = train_df['question_text'].apply(strip_html)

In [ ]:
STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    data = data.dropna(how="any")
    data["question_text"] = data["question_text"].apply(clean_sentence)
    
    return data

#Work_in_Progress